# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bangar,16.8936,120.4243,21.21,67,96,1.87,PH,1706379493
1,1,san patricio,28.0170,-97.5169,19.39,60,42,7.86,US,1706379493
2,2,amurzet,47.6967,131.0981,-24.18,99,59,0.93,RU,1706379493
3,3,thompson,55.7435,-97.8558,-11.16,95,100,2.15,CA,1706379493
4,4,isafjordur,66.0755,-23.1240,-1.85,54,59,8.91,IS,1706379494


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
humid_map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity",
    scale = 0.5, 
    tiles="OSM", 
    color="City", 
    color_palette = "viridis", 
    alpha=0.75, 
    frame_width = 700,
    frame_height = 500  
)

# Display the map
humid_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_vacation_df = city_data_df.loc[(city_data_df['Max Temp'] < 30) & 
                                     (city_data_df['Max Temp'] > 22) &
                                     (city_data_df['Humidity'] < 70) &
                                     (city_data_df['Cloudiness'] < 20)]

# Drop any rows with null values
ideal_vacation_df.dropna()
# Display sample data
ideal_vacation_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,taoudenni,22.6783,-3.9836,23.55,11,0,4.57,ML,1706379498
66,66,castillos,-34.1667,-53.8333,25.88,48,0,6.06,UY,1706379511
76,76,thinadhoo,0.5333,72.9333,28.23,69,4,5.40,MV,1706379514
130,130,baissa,7.2333,10.6333,28.90,18,0,1.67,NG,1706379527
134,134,tazacorte,28.6290,-17.9293,22.54,43,0,6.17,ES,1706379528
144,144,ixtapa,20.7000,-105.2000,25.57,61,0,0.00,MX,1706379340
234,234,ayorou,14.7308,0.9174,24.60,15,1,5.00,NE,1706379556
252,252,cafayate,-26.0730,-65.9761,28.16,44,13,4.67,AR,1706379561
253,253,lazaro cardenas,17.9583,-102.2000,29.47,46,2,2.13,MX,1706379562
255,255,cabo san lucas,22.8909,-109.9124,26.95,41,0,1.54,MX,1706379562


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame({"City": ideal_vacation_df["City"], 
            "Country": ideal_vacation_df["Country"], 
             "Lat": ideal_vacation_df["Lat"], 
             "Lng": ideal_vacation_df["Lng"], 
             "Humidity": ideal_vacation_df["Humidity"], 
            })

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ' '

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
21,taoudenni,ML,22.6783,-3.9836,11,
66,castillos,UY,-34.1667,-53.8333,48,
76,thinadhoo,MV,0.5333,72.9333,69,
130,baissa,NG,7.2333,10.6333,18,
134,tazacorte,ES,28.6290,-17.9293,43,
144,ixtapa,MX,20.7000,-105.2000,61,
234,ayorou,NE,14.7308,0.9174,15,
252,cafayate,AR,-26.0730,-65.9761,44,
253,lazaro cardenas,MX,17.9583,-102.2000,46,
255,cabo san lucas,MX,22.8909,-109.9124,41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1, 
    "filter": " ", 
    "bias": " ",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
taoudenni - nearest hotel: No hotel found
castillos - nearest hotel: A mi gente
thinadhoo - nearest hotel: No hotel found
baissa - nearest hotel: Chap
tazacorte - nearest hotel: App Leyma
ixtapa - nearest hotel: Ma. Cristina
ayorou - nearest hotel: Hotel Aminokal
cafayate - nearest hotel: Il Cavallino
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
cabo san lucas - nearest hotel: Comfort Rooms
toungo - nearest hotel: No hotel found
moissala - nearest hotel: No hotel found
saint-raphael - nearest hotel: No hotel found
yei - nearest hotel: New Tokyo
san jose de feliciano - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: Hotel del Valle
zambezi - nearest hotel: No hotel found
bitam - nearest hotel: Hôtel du stade
yorosso - nearest hotel: No hotel found
san jeronimo de juarez - nearest hotel: No hotel found
ugep - nearest hotel: No hotel found
kampong thom - nearest hotel: សណ្ឋាគារអរុណរះ


,City,Country,Lat,Lng,Humidity,Hotel Name
21,taoudenni,ML,22.6783,-3.9836,11,No hotel found
66,castillos,UY,-34.1667,-53.8333,48,A mi gente
76,thinadhoo,MV,0.5333,72.9333,69,No hotel found
130,baissa,NG,7.2333,10.6333,18,Chap
134,tazacorte,ES,28.6290,-17.9293,43,App Leyma
144,ixtapa,MX,20.7000,-105.2000,61,Ma. Cristina
234,ayorou,NE,14.7308,0.9174,15,Hotel Aminokal
252,cafayate,AR,-26.0730,-65.9761,44,Il Cavallino
253,lazaro cardenas,MX,17.9583,-102.2000,46,Hotel Sol del Pacífico
255,cabo san lucas,MX,22.8909,-109.9124,41,Comfort Rooms


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    scale = 0.5, 
    tiles="OSM", 
    hover_cols = ['Hotel Name', 'Country'], 
    color="City", 
    color_palette = "viridis", 
    alpha=0.75, 
    frame_width = 700,
    frame_height = 500
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)